In [1]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tensorflow.keras.utils import pad_sequences
from collections import Counter
from pandarallel import pandarallel
import pandas as pd
import numpy as np

2023-08-12 23:28:43.995742: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-12 23:28:44.066222: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-12 23:28:45.439625: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
seq = "sunshine birds sing sky blue flowers bloom gently over green fields".split()

In [60]:
vocabulary = set((word, ) for word in seq)
vocabulary

{('birds',),
 ('bloom',),
 ('blue',),
 ('fields',),
 ('flowers',),
 ('gently',),
 ('green',),
 ('over',),
 ('sing',),
 ('sky',),
 ('sunshine',)}

In [61]:
n_grams = set(list(ngrams(seq, 2)) + list(ngrams(seq, 3)))

In [63]:
tokens = sorted(list(vocabulary.union(n_grams)), key=len, reverse=True)
tokens

[('birds', 'sing', 'sky'),
 ('flowers', 'bloom', 'gently'),
 ('sky', 'blue', 'flowers'),
 ('bloom', 'gently', 'over'),
 ('gently', 'over', 'green'),
 ('sunshine', 'birds', 'sing'),
 ('over', 'green', 'fields'),
 ('blue', 'flowers', 'bloom'),
 ('sing', 'sky', 'blue'),
 ('sing', 'sky'),
 ('birds', 'sing'),
 ('flowers', 'bloom'),
 ('green', 'fields'),
 ('gently', 'over'),
 ('over', 'green'),
 ('sunshine', 'birds'),
 ('blue', 'flowers'),
 ('bloom', 'gently'),
 ('sky', 'blue'),
 ('sing',),
 ('flowers',),
 ('fields',),
 ('sky',),
 ('gently',),
 ('birds',),
 ('sunshine',),
 ('bloom',),
 ('over',),
 ('green',),
 ('blue',)]

In [67]:
token_dict = {
    token : i for i, token in enumerate(tokens)
}
token_dict

{('birds', 'sing', 'sky'): 0,
 ('flowers', 'bloom', 'gently'): 1,
 ('sky', 'blue', 'flowers'): 2,
 ('bloom', 'gently', 'over'): 3,
 ('gently', 'over', 'green'): 4,
 ('sunshine', 'birds', 'sing'): 5,
 ('over', 'green', 'fields'): 6,
 ('blue', 'flowers', 'bloom'): 7,
 ('sing', 'sky', 'blue'): 8,
 ('sing', 'sky'): 9,
 ('birds', 'sing'): 10,
 ('flowers', 'bloom'): 11,
 ('green', 'fields'): 12,
 ('gently', 'over'): 13,
 ('over', 'green'): 14,
 ('sunshine', 'birds'): 15,
 ('blue', 'flowers'): 16,
 ('bloom', 'gently'): 17,
 ('sky', 'blue'): 18,
 ('sing',): 19,
 ('flowers',): 20,
 ('fields',): 21,
 ('sky',): 22,
 ('gently',): 23,
 ('birds',): 24,
 ('sunshine',): 25,
 ('bloom',): 26,
 ('over',): 27,
 ('green',): 28,
 ('blue',): 29}

In [65]:
lengths_of_tokens = sorted(set(len(token) for token in tokens), reverse=True)
for n_gram_length in lengths_of_tokens: 
    for win in 
    

[3, 2, 1]